In [29]:
import pandas as pd
import numpy as np
domain_df = pd.read_csv("../../data/initial_data.csv")
domain_df['new_domain'] = 'www.'+ domain_df.domain
domain_df.head(10)

,domain,domain_type,new_domain
0,wisuolycossttqrj.com,dga,www.wisuolycossttqrj.com
1,wi-wamss.org,benign,www.wi-wamss.org
2,qcxfurnkbqidxxcl.biz,dga,www.qcxfurnkbqidxxcl.biz
3,192-168-1-1-admin.ru,benign,www.192-168-1-1-admin.ru
4,dblsiobnkjxomkmh.ru,dga,www.dblsiobnkjxomkmh.ru
5,su5aksay.ru,benign,www.su5aksay.ru
6,sisalex.edu.eg,benign,www.sisalex.edu.eg
7,vloiolycossttqrj.cn,dga,www.vloiolycossttqrj.cn
8,bmabazar.com,benign,www.bmabazar.com
9,msyxwioqtcpbbypd.info,dga,www.msyxwioqtcpbbypd.info


In [30]:
# Creating feature
#Transforming each character from the domain to vowels.

def transform(val):
    vowel = ('a','e','i','o','u')
    new = ''
    for char in val:
        if char in vowel:
            new += 'v'
        elif char.isalpha() and char not in vowel:
            new += 'c'
        elif char.isdigit():
            new += 'n'
        else:
            new += 's'
    return new
domain_df['abbreviation']= domain_df['new_domain'].apply(transform)
domain_df.head(10)

,domain,domain_type,new_domain,abbreviation
0,wisuolycossttqrj.com,dga,www.wisuolycossttqrj.com,cccscvcvvcccvcccccccscvc
1,wi-wamss.org,benign,www.wi-wamss.org,cccscvscvcccsvcc
2,qcxfurnkbqidxxcl.biz,dga,www.qcxfurnkbqidxxcl.biz,cccsccccvcccccvcccccscvc
3,192-168-1-1-admin.ru,benign,www.192-168-1-1-admin.ru,cccsnnnsnnnsnsnsvccvcscv
4,dblsiobnkjxomkmh.ru,dga,www.dblsiobnkjxomkmh.ru,cccsccccvvcccccvccccscv
5,su5aksay.ru,benign,www.su5aksay.ru,cccscvnvccvcscv
6,sisalex.edu.eg,benign,www.sisalex.edu.eg,cccscvcvcvcsvcvsvc
7,vloiolycossttqrj.cn,dga,www.vloiolycossttqrj.cn,cccsccvvvcccvcccccccscc
8,bmabazar.com,benign,www.bmabazar.com,cccsccvcvcvcscvc
9,msyxwioqtcpbbypd.info,dga,www.msyxwioqtcpbbypd.info,cccscccccvvcccccccccsvccv


In [31]:
from nltk.util import ngrams
def extract_ngrams(row, num):
    n_grams = ngrams(row, num)
    return [ ''.join(grams) for grams in n_grams]
for i in range(1,4):
    domain_df['{}gram'.format(i)]= domain_df['abbreviation'].apply(extract_ngrams, args=(i,))
domain_df.head()

,domain,domain_type,new_domain,abbreviation,1gram,2gram,3gram
0,wisuolycossttqrj.com,dga,www.wisuolycossttqrj.com,cccscvcvvcccvcccccccscvc,"[c, c, c, s, c, v, c, v, v, c, c, c, v, c, c, ...","[cc, cc, cs, sc, cv, vc, cv, vv, vc, cc, cc, c...","[ccc, ccs, csc, scv, cvc, vcv, cvv, vvc, vcc, ..."
1,wi-wamss.org,benign,www.wi-wamss.org,cccscvscvcccsvcc,"[c, c, c, s, c, v, s, c, v, c, c, c, s, v, c, c]","[cc, cc, cs, sc, cv, vs, sc, cv, vc, cc, cc, c...","[ccc, ccs, csc, scv, cvs, vsc, scv, cvc, vcc, ..."
2,qcxfurnkbqidxxcl.biz,dga,www.qcxfurnkbqidxxcl.biz,cccsccccvcccccvcccccscvc,"[c, c, c, s, c, c, c, c, v, c, c, c, c, c, v, ...","[cc, cc, cs, sc, cc, cc, cc, cv, vc, cc, cc, c...","[ccc, ccs, csc, scc, ccc, ccc, ccv, cvc, vcc, ..."
3,192-168-1-1-admin.ru,benign,www.192-168-1-1-admin.ru,cccsnnnsnnnsnsnsvccvcscv,"[c, c, c, s, n, n, n, s, n, n, n, s, n, s, n, ...","[cc, cc, cs, sn, nn, nn, ns, sn, nn, nn, ns, s...","[ccc, ccs, csn, snn, nnn, nns, nsn, snn, nnn, ..."
4,dblsiobnkjxomkmh.ru,dga,www.dblsiobnkjxomkmh.ru,cccsccccvvcccccvccccscv,"[c, c, c, s, c, c, c, c, v, v, c, c, c, c, c, ...","[cc, cc, cs, sc, cc, cc, cc, cv, vv, vc, cc, c...","[ccc, ccs, csc, scc, ccc, ccc, ccv, cvv, vvc, ..."


In [32]:
from collections import Counter
def generate_combinations(n, comb):
    if len(comb) == n:
        res.add(''.join(comb))
        return
    for char in ['c','v','s','n']:
        generate_combinations(n,comb+[char])
def bag_of_words_mapper(row, col):
    ngramlist = list(row)
    c = Counter(ngramlist)
    return c[col]
count = 1
for i in range(1,4):
    res = set()
    generate_combinations(i,[])
    for col in res:
        domain_df[col] =  domain_df['{}gram'.format(i)].apply(bag_of_words_mapper, args=(col,))
    df_ = domain_df[list(res)]
    domain_df['{}gram mean'.format(i)] = df_.mean(axis=1)
    domain_df['{}gram variance'.format(i)] = df_.var(axis=1)
    domain_df['{}gram std'.format(i)] = df_.std(axis=1)    

In [35]:
domain_df['F15'] = domain_df['new_domain'].apply(lambda x : len(set(x)))
domain_df['F16'] = domain_df['new_domain'].apply(lambda x : x.count('n')/len(x))
domain_df['F17'] = domain_df['new_domain'].apply(lambda x : x.count('c')/len(x))
domain_df['F18'] = domain_df['new_domain'].apply(lambda x : x.count('c')/x.count('v') if x.count('v') != 0 else x.count('c'))

In [41]:
X = domain_df.drop(['domain','1gram','2gram','3gram','abbreviation','new_domain'], axis=1)

In [42]:
y = X.pop('domain_type')

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(X_train, y_train)
y_predict =model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.9965
              precision    recall  f1-score   support

      benign       1.00      0.99      1.00      1978
         dga       0.99      1.00      1.00      2022

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



col_0,benign,dga
domain_type,,
benign,1964,14
dga,0,2022


In [45]:
# tuning
model2=XGBClassifier(max_depth=2, learning_rate =0.07, random_state =575, n_estimators=200)
model2.fit(X_train, y_train)
y_predict =model2.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.9955
              precision    recall  f1-score   support

      benign       1.00      0.99      1.00      1978
         dga       0.99      1.00      1.00      2022

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



col_0,benign,dga
domain_type,,
benign,1960,18
dga,0,2022


In [48]:
# use Boruta
from boruta import BorutaPy
xgb=XGBClassifier(max_depth=2, learning_rate =0.07, random_state =575, n_estimators=200)
feat_selector = BorutaPy(xgb, n_estimators='auto', verbose=2, random_state=1)
feat_selector.fit(X_train.values, y_train.values)
feat_selector.support_
feat_selector.ranking_

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	97
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	9 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	10 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	11 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	12 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	13 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	14 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	15 / 100
Confirmed: 	22
Tentative: 	4
Rejected: 	71
Iteration: 	16 / 100
Confirmed: 	23
Tentative: 	3
Reject

array([ 1, 40, 40,  1,  1,  1, 40, 40, 40, 40, 40,  5, 40, 40, 40,  1,  2,
       40, 40,  1, 40,  1,  1, 40,  1, 40, 40,  1, 40, 40, 40, 40, 40, 40,
        1,  1, 40, 40, 40, 40, 40,  1, 40, 40, 40, 40, 40,  4, 40, 40, 40,
        2, 40, 40, 40, 40, 40, 40, 40, 40, 40,  1,  3, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,  1, 40, 40,  1,  1,
        1, 40, 40, 40,  6, 40,  1, 40,  1,  1,  1,  1])

In [49]:
X_filtered_train = feat_selector.transform(X_train.values)
X_filtered_test = feat_selector.transform(X_test.values)

In [50]:
xgb_bo=XGBClassifier(max_depth=2, learning_rate =0.07, random_state =575, n_estimators=200)
xgb_bo.fit(X_filtered_train, y_train)
y_predict =xgb_bo.predict(X_filtered_test)
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.9955
              precision    recall  f1-score   support

      benign       1.00      0.99      1.00      1978
         dga       0.99      1.00      1.00      2022

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



col_0,benign,dga
domain_type,,
benign,1960,18
dga,0,2022


In [51]:
X_filtered_test.shape
X_filtered_train.shape

(16000, 23)

In [52]:
X_train.columns

Index(['s', 'n', 'c', 'v', '1gram mean', '1gram variance', '1gram std', 'sn',
       'cs', 'nc', 'ns', 'sc', 'cn', 'vn', 'nv', 'cv', 'vc', 'sv', 'ss', 'cc',
       'nn', 'vv', 'vs', '2gram mean', '2gram variance', '2gram std', 'snn',
       'vcc', 'vcn', 'scs', 'svs', 'nnv', 'csv', 'nnc', 'vcv', 'ccs', 'svc',
       'css', 'cvn', 'nvn', 'ncs', 'vvc', 'csn', 'vvs', 'vnv', 'vsn', 'nsv',
       'ccv', 'ncn', 'vnc', 'vnn', 'svv', 'ssv', 'ssn', 'cnn', 'ncc', 'nvv',
       'nsn', 'sss', 'nvs', 'cnc', 'cvc', 'vcs', 'nsc', 'cnv', 'vvn', 'vsc',
       'csc', 'vns', 'svn', 'cns', 'nnn', 'nns', 'nvc', 'vss', 'ncv', 'vsv',
       'scn', 'ccn', 'snv', 'cvv', 'cvs', 'ssc', 'ccc', 'scv', 'scc', 'snc',
       'sns', 'nss', 'vvv', '3gram mean', '3gram variance', '3gram std', 'F15',
       'F16', 'F17', 'F18'],
      dtype='object')

In [55]:
res = set()
def generate_combinations(n, comb):
    if len(comb) == n:
        res.add(''.join(comb))
        return
    for char in ['c','v','s','n']:
        generate_combinations(n,comb+[char])
generate_combinations(2, [])
print(res)

{'sn', 'cs', 'nc', 'ns', 'sc', 'cn', 'vn', 'nv', 'cv', 'vc', 'sv', 'ss', 'cc', 'nn', 'vv', 'vs'}


In [31]:
assignment = pd.read_csv("assignment.csv")
assignment['feature']= assignment['domain'].apply(transform)
assignment

,domain,feature
0,www.dutchwebdesign.com,cccscvccccvccvcvccscvc
1,www.pploulsosohvmkv.com,cccscccvvccvcvcccccscvc
2,www.cfsjdqwdhmwkiv.com,cccsccccccccccccvcscvc
3,www.uocoqgiusyeciouaimcauykqswsymo.com,cccsvvcvccvvccvcvvvvvccvvccccccccvscvc
4,www.total-toolbar.com,cccscvcvcscvvccvcscvc
...,...,...
495,www.u23lib5bcxw0yju.com,cccsvnncvcnccccnccvscvc
496,www.prmysangh.com,cccscccccvcccscvc
497,www.ociqkeasqocgoecksu.com,cccsvcvccvvccvccvvcccvscvc
498,www.hj-9.com,cccsccsnscvc


In [67]:
def bagofwords(assignment):
    df = pd.DataFrame(data=assignment.feature)
    for i in assignment.feature:
        #1-gram
        df.loc[df['feature'] == i ,'v'] =i.count('v')
        df.loc[df['feature'] == i ,'c'] =i.count('c')
        df.loc[df['feature'] == i ,'n'] =i.count('n')
        df.loc[df['feature'] == i ,'s'] =i.count('s')

        # 2-gram
        df.loc[df['feature'] == i ,'vv'] =i.count('vv')
        df.loc[df['feature'] == i ,'vc'] =i.count('vc')
        df.loc[df['feature'] == i ,'vn'] =i.count('vn')
        df.loc[df['feature'] == i ,'vs'] =i.count('vs')
        df.loc[df['feature'] == i ,'cv'] =i.count('cv')
        df.loc[df['feature'] == i ,'cc'] =i.count('cc')
        df.loc[df['feature'] == i ,'cn'] =i.count('cn')
        df.loc[df['feature'] == i ,'cs'] =i.count('cs')
        df.loc[df['feature'] == i ,'nv'] =i.count('nv')
        df.loc[df['feature'] == i ,'nc'] =i.count('nc')
        df.loc[df['feature'] == i ,'nn'] =i.count('nn')
        df.loc[df['feature'] == i ,'ns'] =i.count('ns')
        df.loc[df['feature'] == i ,'sv'] =i.count('sv')
        df.loc[df['feature'] == i ,'sc'] =i.count('sc')
        df.loc[df['feature'] == i ,'sn'] =i.count('sn')
        df.loc[df['feature'] == i ,'ss'] =i.count('ss')
         # 3-gram
        df.loc[df['feature'] == i ,'vvv'] =i.count('vvv')
        df.loc[df['feature'] == i ,'vcv'] =i.count('vcv')
        df.loc[df['feature'] == i ,'vnv'] =i.count('vnv')
        df.loc[df['feature'] == i ,'vsv'] =i.count('vsv')
        df.loc[df['feature'] == i ,'cvv'] =i.count('cvv')
        df.loc[df['feature'] == i ,'ccv'] =i.count('ccv')
        df.loc[df['feature'] == i ,'cnv'] =i.count('cnv')
        df.loc[df['feature'] == i ,'csv'] =i.count('csv')
        df.loc[df['feature'] == i ,'nvv'] =i.count('nvv')
        df.loc[df['feature'] == i ,'ncv'] =i.count('ncv')
        df.loc[df['feature'] == i ,'nnv'] =i.count('nnv')
        df.loc[df['feature'] == i ,'nsv'] =i.count('nsv')
        df.loc[df['feature'] == i ,'svv'] =i.count('svv')
        df.loc[df['feature'] == i ,'scv'] =i.count('scv')
        df.loc[df['feature'] == i ,'snv'] =i.count('snv')
        df.loc[df['feature'] == i ,'ssv'] =i.count('ssv')
        df.loc[df['feature'] == i ,'vvc'] =i.count('vvc')
        df.loc[df['feature'] == i ,'vcc'] =i.count('vcc')
        df.loc[df['feature'] == i ,'vnc'] =i.count('vnc')
        df.loc[df['feature'] == i ,'vsc'] =i.count('vsc')
        df.loc[df['feature'] == i ,'cvc'] =i.count('cvc')
        df.loc[df['feature'] == i ,'ccc'] =i.count('ccc')
        df.loc[df['feature'] == i ,'cnc'] =i.count('cnc')
        df.loc[df['feature'] == i ,'csc'] =i.count('csc')
        df.loc[df['feature'] == i ,'nvc'] =i.count('nvc')
        df.loc[df['feature'] == i ,'ncc'] =i.count('ncc')
        df.loc[df['feature'] == i ,'nnc'] =i.count('nnc')
        df.loc[df['feature'] == i ,'nsc'] =i.count('nsc')
        df.loc[df['feature'] == i ,'svc'] =i.count('svc')
        df.loc[df['feature'] == i ,'scc'] =i.count('scc')
        df.loc[df['feature'] == i ,'snc'] =i.count('snc')
        df.loc[df['feature'] == i ,'ssc'] =i.count('ssc')
        df.loc[df['feature'] == i ,'vvn'] =i.count('vvn')
        df.loc[df['feature'] == i ,'vcn'] =i.count('vcn')
        df.loc[df['feature'] == i ,'vnn'] =i.count('vnn')
        df.loc[df['feature'] == i ,'vsn'] =i.count('vsn')
        df.loc[df['feature'] == i ,'cvn'] =i.count('cvn')
        df.loc[df['feature'] == i ,'ccn'] =i.count('ccn')
        df.loc[df['feature'] == i ,'cnn'] =i.count('cnn')
        df.loc[df['feature'] == i ,'csn'] =i.count('csn')
        df.loc[df['feature'] == i ,'nvn'] =i.count('nvn')
        df.loc[df['feature'] == i ,'ncn'] =i.count('ncn')
        df.loc[df['feature'] == i ,'nnn'] =i.count('nnn')
        df.loc[df['feature'] == i ,'nsn'] =i.count('nsn')
        df.loc[df['feature'] == i ,'svn'] =i.count('svn')
        df.loc[df['feature'] == i ,'scn'] =i.count('scn')
        df.loc[df['feature'] == i ,'snn'] =i.count('snn')
        df.loc[df['feature'] == i ,'ssn'] =i.count('ssn')
        df.loc[df['feature'] == i ,'vvs'] =i.count('vvs')
        df.loc[df['feature'] == i ,'vcs'] =i.count('vcs')
        df.loc[df['feature'] == i ,'vns'] =i.count('vns')
        df.loc[df['feature'] == i ,'vss'] =i.count('vss')
        df.loc[df['feature'] == i ,'cvs'] =i.count('cvs')
        df.loc[df['feature'] == i ,'ccs'] =i.count('ccs')
        df.loc[df['feature'] == i ,'cns'] =i.count('cns')
        df.loc[df['feature'] == i ,'css'] =i.count('css')
        df.loc[df['feature'] == i ,'nvs'] =i.count('nvs')
        df.loc[df['feature'] == i ,'ncs'] =i.count('ncs')
        df.loc[df['feature'] == i ,'nns'] =i.count('nns')
        df.loc[df['feature'] == i ,'nss'] =i.count('nss')
        df.loc[df['feature'] == i ,'svs'] =i.count('svs')
        df.loc[df['feature'] == i ,'scs'] =i.count('scs')
        df.loc[df['feature'] == i ,'sns'] =i.count('sns')
        df.loc[df['feature'] == i ,'sss'] =i.count('sss')

        #other features
        df.loc[df['feature'] == i ,'length']= len(i)
        df.loc[df['feature'] == i ,'uniqueChar']= len(set(assignment.loc[assignment['feature'] == i].domain.to_string()))
    return df.drop(['feature'], axis=1)


In [69]:
bagofwords(assignment).head()

,v,c,n,s,vv,vc,vn,vs,cv,cc,...,nvs,ncs,nns,nss,svs,scs,sns,sss,length,uniqueChar
0,5.0,15.0,0.0,2.0,0.0,5.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,17.0
1,5.0,16.0,0.0,2.0,1.0,4.0,0.0,0.0,4.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,14.0
2,2.0,18.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,16.0
3,15.0,21.0,0.0,2.0,5.0,7.0,0.0,1.0,7.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,16.0
4,6.0,12.0,0.0,3.0,1.0,5.0,0.0,0.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,13.0


In [71]:
assignment_predict =model.predict(bagofwords(assignment))

In [72]:
assignment['threat']= assignment_predict

In [73]:
assignment.threat.value_counts()

benign    466
dga        34
Name: threat, dtype: int64

In [74]:
assignment.head()

,domain,feature,threat
0,www.dutchwebdesign.com,cccscvccccvccvcvccscvc,benign
1,www.pploulsosohvmkv.com,cccscccvvccvcvcccccscvc,benign
2,www.cfsjdqwdhmwkiv.com,cccsccccccccccccvcscvc,benign
3,www.uocoqgiusyeciouaimcauykqswsymo.com,cccsvvcvccvvccvcvvvvvccvvccccccccvscvc,benign
4,www.total-toolbar.com,cccscvcvcscvvccvcscvc,benign


In [75]:
assignment2 = assignment.drop('feature',axis=1)
assignment2.to_csv('assignment_output.csv', index=False)